In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# pip install seaborn
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Data Cleaning

In [10]:
dfClean = pd.read_csv('data/cleaned_merged_seasons.csv')
pd.set_option("display.max_columns", None)
dfClean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98402 entries, 0 to 98401
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         98402 non-null  int64  
 1   season_x           98402 non-null  object 
 2   name               98402 non-null  object 
 3   position           98402 non-null  object 
 4   team_x             48930 non-null  object 
 5   assists            98402 non-null  int64  
 6   bonus              98402 non-null  int64  
 7   bps                98402 non-null  int64  
 8   clean_sheets       98402 non-null  int64  
 9   creativity         98402 non-null  float64
 10  element            98402 non-null  int64  
 11  fixture            98402 non-null  int64  
 12  goals_conceded     98402 non-null  int64  
 13  goals_scored       98402 non-null  int64  
 14  ict_index          98402 non-null  float64
 15  influence          98402 non-null  float64
 16  kickoff_time       984

C:\Users\22352155\AppData\Local\Temp\ipykernel_21748\2754541814.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfClean = pd.read_csv('data/cleaned_merged_seasons.csv')


In [12]:
dfClean.shape

(98402, 38)

In [16]:
print('We have {} NaN/Null values in this dataset'.format(dfClean.isnull().values.sum()))

We have 49472 NaN/Null values in this dataset


In [14]:
print(98402-48930)

49472


There are 98 missing values in the two columns: 'team_a_score' and 'team_h_score'.
We will  now set them to 0.

In [11]:
dfClean['team_a_score'] = dfClean['team_a_score'].fillna(0)
dfClean['team_h_score'] = dfClean['team_h_score'].fillna(0)
dfClean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98402 entries, 0 to 98401
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         98402 non-null  int64  
 1   season_x           98402 non-null  object 
 2   name               98402 non-null  object 
 3   position           98402 non-null  object 
 4   team_x             48930 non-null  object 
 5   assists            98402 non-null  int64  
 6   bonus              98402 non-null  int64  
 7   bps                98402 non-null  int64  
 8   clean_sheets       98402 non-null  int64  
 9   creativity         98402 non-null  float64
 10  element            98402 non-null  int64  
 11  fixture            98402 non-null  int64  
 12  goals_conceded     98402 non-null  int64  
 13  goals_scored       98402 non-null  int64  
 14  ict_index          98402 non-null  float64
 15  influence          98402 non-null  float64
 16  kickoff_time       984

In [17]:
pd.set_option("display.max_columns", None)
dfClean.head(6)

,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,10,0,0,0.0,0.0,2016-08-15T19:00:00Z,0,4,Chelsea,0,0,0,0,1,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,3,0,0,0.9,8.2,2016-08-13T14:00:00Z,15,17,Spurs,0,0,0,0,1,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,8,3,0,3.0,2.2,2016-08-14T15:00:00Z,60,9,Liverpool,0,0,0,0,1,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,7,0,0,0.0,0.0,2016-08-13T14:00:00Z,0,13,Southampton,0,0,0,0,1,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,6,1,0,0.3,2.0,2016-08-13T14:00:00Z,69,14,Stoke,0,0,0,0,1,0,2723,1.0,1.0,0.0,1,0,0,0,45,True,1,1
5,5,2016-17,Adam Lallana,MID,NaN,1,2,33,0,33.7,205,8,3,1,14.2,51.2,2016-08-14T15:00:00Z,75,1,Arsenal,0,0,0,0,1,0,155525,4.0,3.0,57.0,11,0,0,0,70,False,1,1


In [19]:
# Taking away all the irrelevant features
featuresToKeep = list(
    set(dfClean.columns)
    - {
        "team_x",
        "opponent_team",
        "opp_team_name",
        "was_home",
        "team_a_score",
        "round",
        "fixture",
        "element",
        "saves",
        "team_h_score",
        "own_goals",
        "saves",
        "opponent_team",
        "penalties_missed",
        "penalties_saved",
        "red_cards",
        "kickoff_time",
        "bps",
        "assists",
        "bonus",
        "clean_sheets",
        "goals_conceded",
        "yellow_cards",
        "transfers_balance"
    }
)

dfRemoved =dfClean[featuresToKeep]

dfRemoved.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98402 entries, 0 to 98401
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season_x       98402 non-null  object 
 1   transfers_out  98402 non-null  int64  
 2   total_points   98402 non-null  int64  
 3   minutes        98402 non-null  int64  
 4   transfers_in   98402 non-null  int64  
 5   name           98402 non-null  object 
 6   selected       98402 non-null  int64  
 7   Unnamed: 0     98402 non-null  int64  
 8   value          98402 non-null  int64  
 9   position       98402 non-null  object 
 10  ict_index      98402 non-null  float64
 11  creativity     98402 non-null  float64
 12  GW             98402 non-null  int64  
 13  goals_scored   98402 non-null  int64  
 14  threat         98402 non-null  float64
 15  influence      98402 non-null  float64
dtypes: float64(4), int64(9), object(3)
memory usage: 12.0+ MB
